In [97]:
#In this algorithm we created dictionary that works backwards, 1->Arsenal ,...
#we did it by inversing the dictionary we used to set the team names to numeric
#then what we do is we let the user to choose what team he wants to try and predict their home games attendance
#here we also let the user choose between two linear regression models
#one by one variable
#second by multiple variables
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model
from sklearn import metrics
%matplotlib inline

In [98]:
df = pd.read_csv('data_ready_for_ML_1.csv')
teams_to_num_df = pd.read_csv('teams_dict_data_frame.csv')

In [99]:
def give_me_from_this_df_dictionary(teams_to_num_df):
    team_number = teams_to_num_df['team_number']
    team_name = teams_to_num_df['team_name']
    return {team_number[i]: team_name[i] for i in range(len(team_number))}

In [100]:
teams_dictionary = give_me_from_this_df_dictionary(teams_to_num_df)

In [101]:
#fixing the dent
teams_dictionary[0]='Empty dataset'

In [102]:
data_frames_list = list()

In [103]:
for x in sorted(teams_dictionary):
    new_data_frame = df[df["team_1"] == x]
    new_data_frame = new_data_frame.iloc[:,1:]
    data_frames_list.append(new_data_frame)

In [104]:
#Reversing the dictionary:
inv_teams_nums_dict = {v: k for k, v in teams_dictionary.items()}

In [105]:
def give_me_this_team_plot(this_data):
    this_data.plot(x='year', y='attendance', style='o')  
    plt.title('year vs attendance')  
    plt.xlabel('year')  
    plt.ylabel('attendance')  
    plt.show()

In [106]:
def give_me_this_team_histogram(this_data):
    plt.figure(figsize=(15,10))
    plt.tight_layout()
    seabornInstance.distplot(this_data['attendance'])

In [107]:
def predict_using_lr_one_var(this_team, inv_teams_nums_dict, data_frames_list):
    #lr_rating = linear_model.LinearRegression() 
    
    current_data_frame=data_frames_list[inv_teams_nums_dict[this_team]]
    
    X = current_data_frame['year'].values.reshape(-1,1)
    y = current_data_frame['attendance'].values.reshape(-1,1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = linear_model.LinearRegression()  
    regressor.fit(X_train, y_train) #training the algorithm
    #To retrieve the intercept:
    print(regressor.intercept_)

    #For retrieving the slope:
    print(regressor.coef_)
    
    y_pred = regressor.predict(X_test)

    prediction_dataframe = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
    draw_scatter_plot_equality(X_test, y_test, y_pred)
    #draw_bar_equality(prediction_dataframe)
    return prediction_dataframe
    

In [108]:
def predict_using_lr_two_var(this_team, inv_teams_nums_dict, data_frames_list):
    #lr_rating = linear_model.LinearRegression() 
    
    current_data_frame=data_frames_list[inv_teams_nums_dict[this_team]]
    
    #current_data_frame = current_data_frame[(current_data_frame['year'] != 1995) & (current_data_frame['year'] != 1996) & (current_data_frame['year'] != 1997) & (current_data_frame['year'] != 1998) & (current_data_frame['year'] != 1999)]
    
    X = current_data_frame[["team_1_rating","team_2_rating", "month", "derby", "city", "year", "temprature", "precipitation" , "capacity"]]
    y = current_data_frame['attendance']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = linear_model.LinearRegression()  
    regressor.fit(X_train, y_train) #training the algorithm
    #To retrieve the intercept:
    print(regressor.intercept_)

    #For retrieving the slope:
    print(regressor.coef_)
    
    y_pred = regressor.predict(X_test)

    prediction_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    #draw_scatter_plot_equality(X_test, y_test, y_pred)
    #draw_bar_equality(prediction_dataframe)
    return prediction_dataframe
    

In [109]:
def draw_bar_equality(this_data):
    this_data_1 = this_data.head(25)
    this_data_1.plot(kind='bar',figsize=(16,10))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()

In [110]:
def draw_scatter_plot_equality(X_test, y_test, y_pred):
    plt.scatter(X_test, y_test,  color='gray')
    plt.plot(X_test, y_pred, color='red', linewidth=2)
    plt.show()

In [111]:
#Write here the team name you would like to check:
team_name = input("Insert team name: ")
option = int(input("Insert 1 for one variable linear regression / 2 for multiply variables linear regressions: "))
if(option == 1):
    this_df=predict_using_lr_one_var(team_name, inv_teams_nums_dict, data_frames_list)
else:
    this_df=predict_using_lr_two_var(team_name, inv_teams_nums_dict, data_frames_list)

Insert team name: Barnsley
Insert 1 for one variable linear regression / 2 for multiply variables linear regressions: 2
18362.630255411477
[ 22.32322878 -31.9534371   11.66836503   0.           0.
   0.          31.49668343  -7.96686757   0.        ]


In [112]:
this_df.Predicted=this_df['Predicted'].astype('int64')

In [113]:
test = this_df.copy()

In [114]:
test['error'] = 0

In [115]:
test['error_per'] = 0

In [116]:
test

,Actual,Predicted,error,error_per
965,18687,18325,0,0
789,18691,18806,0,0
914,17819,17981,0,0
1131,18667,18515,0,0


In [121]:
for x in range(test.shape[0]):
    current_error = abs(test.iloc[x, 0] - test.iloc[x,1])
    current_error_per = abs(test.iloc[x, 0] - test.iloc[x,1]) / test.iloc[x, 0] * 100
    test.iloc[x,int(test.shape[1])-2] = current_error
    test.iloc[x,int(test.shape[1])-1] = current_error_per

In [122]:
test

,Actual,Predicted,error,error_per
965,18687,18325,362,1.937176
789,18691,18806,115,0.615269
914,17819,17981,162,0.909142
1131,18667,18515,152,0.814271


In [123]:
test.to_csv(team_name+'tested.csv')